#### вход в директорию base_data_pq

In [1]:
cd base_data_pq

C:\laba-python\victory\base_data_pq


#### создание словаря активностей пользователя

In [3]:
import pandas as pd

In [3]:
import csv

actions = {0: "view", 1: "like", 2: "addB", 3: "delB", 4: "clearB", 5: "order", 6: "listB", 7: "visit", 8: "visitCategory", 9: "search"}
csv_file_name = r'C:\laba-python\victory\filtered_data_pq\actions.csv'

with open(csv_file_name, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['action_code', 'action_name'])
    writer.writerows(actions.items())

print(f"CSV файл '{csv_file_name}' успешно создан.")

CSV файл 'C:\laba-python\victory\filtered_data_csv\actions.csv' успешно создан.


#### Требования к train_actions.pq:

    1. Создать отдельный словарь со всеми возможными описаниями user_id к loc_user_id
    2. Учитывать активности 1, 2, 3, 4, 9
    3. Учитывание часто встречающейся активности 0 (значение встретилось больше 2 раз) у уникального пользователя на конкретные товары
    4. Удалить user_id, pageId
    5. Разделить значения date на столбцы day и time
    6. Переименовть products в purchased_items
    7. если в purchased_items будет много значений с [], то лучше их удалить

    тип products - <class 'numpy.ndarray'>

In [9]:
import pandas as pd
import numpy as np

# Загрузка данных из файла
df = pd.read_parquet('train_actions.pq')

# Учитываем все заданные активности (1, 2, 3, 4, 9, 0)
valid_activities = [0, 1, 2, 3, 4, 9]

# Фильтруем данные, оставляя только записи с допустимыми активностями
df_filtered = df[df['action'].isin(valid_activities)]

# Удаление ненужных столбцов
df_filtered = df_filtered.drop(columns=['user_id', 'pageId'])

# Убедитесь, что столбец date в формате datetime
df_filtered['date'] = pd.to_datetime(df_filtered['date'])

# Разделение столбца date на day и time
df_filtered['day'] = df_filtered['date'].dt.date
df_filtered['time'] = df_filtered['date'].dt.time
df_filtered = df_filtered.drop(columns=['date'])  # Удаляем старый столбец date

# Переименование столбца products в purchased_items
df_filtered = df_filtered.rename(columns={'products': 'purchased_items'})

# Удаление строк с пустыми значениями в purchased_items
df_filtered = df_filtered[~df_filtered['purchased_items'].apply(lambda x: isinstance(x, np.ndarray) and len(x) == 0)]

# Сохранение результата в новый файл
df_filtered.to_parquet(r'C:\laba-python\victory\filtered_data_pq\data.pq', index=False)


In [10]:
catalog = pd.read_parquet(r'C:\laba-python\victory\filtered_data_pq\data.pq', engine='pyarrow')
catalog.head(-5)

,loc_user_id,action,purchased_items,day,time
0,b29d3f97-1a02-4e3a-990b-2296935a5479,0,[921892],2024-09-07,00:00:16
1,95badd96-2133-408c-acbf-23acbf3ec808,0,[6344224],2024-09-07,00:01:19
2,95badd96-2133-408c-acbf-23acbf3ec808,0,[6344227],2024-09-07,00:01:33
3,95badd96-2133-408c-acbf-23acbf3ec808,0,[6344221],2024-09-07,00:01:35
4,95badd96-2133-408c-acbf-23acbf3ec808,0,[6344218],2024-09-07,00:01:39
...,...,...,...,...,...
1152221,d2e9c217-aaae-4921-9cae-b79839ed2ef7,1,[6167461],2024-09-28,23:59:26
1152222,1b1c72e6-8841-486e-bfdb-db2647ccfcc2,0,[6389818],2024-09-28,23:59:28
1152223,56498eb8-725d-4c2e-83d8-9b1748e5c2d2,0,[5892955],2024-09-28,23:59:32
1152224,9984a2a2-a611-4e5c-a5ef-6b89e3247875,0,[5392213],2024-09-28,23:59:35


#### Требования к stokman_catalog_preprocessed.pq

    1. Для товаров без описания добавить флаг
    2. Удалить столбцы add_date, shop_id, title, old_price
    3. Товары без price заполнить средним значением или медианой
    4. Товары без category_id заполнить значением 0

In [14]:
import pandas as pd

# Загрузка и обработка данных
df = pd.read_parquet('stokman_catalog_preprocessed.pq')
df.drop(columns=['add_date', 'shop_id', 'title', 'old_price'], inplace=True)
df['price'] = df['price'].fillna(df['price'].mean())  # или df['price'].median()
df['category_id'] = df['category_id'].fillna(0)
df.to_parquet(r'C:\laba-python\victory\filtered_data_pq\catalog.pq')

In [4]:
catalog = pd.read_parquet(r'C:\laba-python\victory\filtered_data_pq\catalog.pq', engine='pyarrow')
catalog.head(-5)

,product_id,category_id,price
0,1025536,8922,6990.0
1,1025614,10600,330.0
2,1025656,231,13240.0
3,1025659,14546,7550.0
4,1025668,1949,1290.0
...,...,...,...
107089,1516589,0,4190.0
107090,7233534,0,4190.0
107091,1577588,0,4190.0
107092,7181874,0,4190.0


##### дополнение каталога с описаниями неуказанными товарами из активностей пользователей

In [18]:
import pandas as pd
import numpy as np

# Загрузка данных из файла filtered_data_pq
filtered_data_path = r'C:\laba-python\victory\filtered_data_pq\data.pq'
catalog_data_path = r'C:\laba-python\victory\filtered_data_pq\catalog.pq'

# Чтение данных из файла
filtered_data = pd.read_parquet(filtered_data_path)
catalog_data = pd.read_parquet(catalog_data_path)

# Получение всех уникальных product_id из purchased_items
unique_product_ids = set()
for items in filtered_data['purchased_items']:
    unique_product_ids.update(items)

# Преобразование product_id из catalog в множество для быстрой проверки наличия
existing_product_ids = set(catalog_data['product_id'])

# Находим отсутствующие product_id и добавляем их в catalog
missing_product_ids = unique_product_ids - existing_product_ids

# Создаем DataFrame для недостающих product_id
if missing_product_ids:
    new_rows = pd.DataFrame({'product_id': list(missing_product_ids)})
    catalog_data = pd.concat([catalog_data, new_rows], ignore_index=True)

# Сохраняем изменения обратно в файл
catalog_data.to_parquet(catalog_data_path, index=False)

print("Обработка завершена! Недостающие product_id добавлены в каталог.")


Обработка завершена! Недостающие product_id добавлены в каталог.


In [26]:
import pandas as pd

# Путь к файлу catalog.pq
catalog_data_path = r'C:\laba-python\victory\filtered_data_pq\catalog.pq'

# Чтение данных из файла
catalog_data = pd.read_parquet(catalog_data_path)

# Замена None в category_id на 0 и преобразование в int
catalog_data['category_id'] = catalog_data['category_id'].fillna(0).astype(int)

# Замена NaN в price на медиану всех цен
median_price = catalog_data['price'].median()
catalog_data['price'] = catalog_data['price'].fillna(median_price)

# Сохраняем изменения обратно в файл
catalog_data.to_parquet(catalog_data_path, index=False)

print("Обработка завершена! Значения category_id и price обновлены.")


Обработка завершена! Значения category_id и price обновлены.


In [27]:
import pandas as pd

# Загрузка данных
df = pd.read_parquet(r'C:\laba-python\victory\filtered_data_pq\catalog.pq')

# Подсчет количества значений category_id, равных 0
count_zero_category = (df['category_id'] == 0).sum()

print(f"Количество category_id со значением 0: {count_zero_category}")


Количество category_id со значением 0: 17631


In [28]:
catalog = pd.read_parquet(r'C:\laba-python\victory\filtered_data_pq\catalog.pq', engine='pyarrow')
catalog.head(-5)

,product_id,category_id,price
0,1025536,8922,6990.0
1,1025614,10600,330.0
2,1025656,231,13240.0
3,1025659,14546,7550.0
4,1025668,1949,1290.0
...,...,...,...
107089,1516589,0,4190.0
107090,7233534,0,4190.0
107091,1577588,0,4190.0
107092,7181874,0,4190.0


# доп. анализ 

    разная длина product_id (7 или 6)
    все принадлежат одному магазину

In [3]:
import pandas as pd

# Путь к файлу catalog.pq
catalog_data_path = r'C:\laba-python\victory\base_data_pq\stokman_catalog_preprocessed.pq'

# Чтение данных из файла
catalog_data = pd.read_parquet(catalog_data_path)

# Предполагаем, что product_id - это строка
# Применяем функцию для определения длины каждого product_id
lengths = catalog_data['product_id'].astype(str).apply(len)

# Определяем максимальную и минимальную длину
max_length = lengths.max()
min_length = lengths.min()

# Подсчет количества значений с 7 цифрами, меньше и больше
count_seven_digits = (lengths == 7).sum()
count_less_than_seven = (lengths < 7).sum()
count_more_than_seven = (lengths > 7).sum()

print(f"Максимальная длина product_id: {max_length}")
print(f"Минимальная длина product_id: {min_length}")
print(f"Количество product_id с 7 цифрами: {count_seven_digits}")
print(f"Количество product_id с меньшим количеством цифр: {count_less_than_seven}")
print(f"Количество product_id с большим количеством цифр: {count_more_than_seven}")


Максимальная длина product_id: 7
Минимальная длина product_id: 6
Количество product_id с 7 цифрами: 82574
Количество product_id с меньшим количеством цифр: 6909
Количество product_id с большим количеством цифр: 0


In [4]:
import pandas as pd

# Путь к файлу catalog.pq
catalog_data_path = r'C:\laba-python\victory\base_data_pq\stokman_catalog_preprocessed.pq'

# Чтение данных из файла
catalog_data = pd.read_parquet(catalog_data_path)

# Предполагаем, что product_id - это строка
# Создаем столбец с длиной каждого product_id
catalog_data['product_id_length'] = catalog_data['product_id'].astype(str).apply(len)

# Получаем shop_id для семизначных product_id
seven_digit_shops = catalog_data[catalog_data['product_id_length'] == 7]['shop_id'].unique()

# Получаем shop_id для шестизначных product_id
six_digit_shops = catalog_data[catalog_data['product_id_length'] == 6]['shop_id'].unique()

print(f"Значения shop_id для семизначных product_id: {seven_digit_shops}")
print(f"Значения shop_id для шестизначных product_id: {six_digit_shops}")


Значения shop_id для семизначных product_id: ['350']
Значения shop_id для шестизначных product_id: ['350']


In [5]:
import pandas as pd

# Путь к файлу catalog.pq
catalog_data_path = r'C:\laba-python\victory\base_data_pq\stokman_catalog_preprocessed.pq'

# Чтение данных из файла
catalog_data = pd.read_parquet(catalog_data_path)

# Проверяем количество нулевых значений в столбце shop_id
null_shop_id_count = catalog_data['shop_id'].isnull().sum()

# Проверяем, есть ли хотя бы одно нулевое значение
has_null_shop_id = null_shop_id_count > 0

print(f"Количество нулевых значений в shop_id: {null_shop_id_count}")
print(f"Содержит ли shop_id нулевые значения? {'Да' if has_null_shop_id else 'Нет'}")


Количество нулевых значений в shop_id: 0
Содержит ли shop_id нулевые значения? Нет


In [6]:
import pandas as pd

# Путь к файлу catalog.pq
catalog_data_path = r'C:\laba-python\victory\base_data_pq\stokman_catalog_preprocessed.pq'

# Чтение данных из файла
catalog_data = pd.read_parquet(catalog_data_path)

# Подсчет количества уникальных shop_id
unique_shop_id_count = catalog_data['shop_id'].nunique()

print(f"Количество уникальных shop_id: {unique_shop_id_count}")


Количество уникальных shop_id: 1
